In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import vectorbt as vbt
import empyrical as ep
import warnings
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Load the data
TFEX_S50 = pd.read_excel("S50_TFEX.xlsx")
TFEX_S50['Timestamp'] = pd.to_datetime(TFEX_S50['Timestamp'])
TFEX_S50.set_index('Timestamp', inplace=True)

set50 = pd.read_csv('SET50.csv')
set50['Timestamp'] = pd.to_datetime(set50['Date'])
set50.set_index('Timestamp', inplace=True)
set50.drop(columns=['Date'], inplace=True)
set50['Close'] = pd.to_numeric(set50['Close'].str.replace(',', ''), errors='coerce')

# Parameters
window = 30
factors = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
cols = TFEX_S50.columns

# List to store results
results = []

# Process each security
for col in cols:
    print(f"Processing security: {col}")

    data = TFEX_S50[[col]].join(set50, how='inner').dropna()
    data[col] = pd.to_numeric(data[col], errors='coerce')
    data['SET50'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Gap'] = data[col] - data['SET50']
    data['Rolling Mean'] = data['Gap'].rolling(window=window).mean()
    data['Rolling Std'] = data['Gap'].rolling(window=window).std()
    data['Z-score'] = (data['Gap'] - data['Rolling Mean']) / data['Rolling Std']
    data['EMA'] = data['Gap'].ewm(span=window, adjust=False).mean()

    # Iterate through each combination of entry and exit Z-score thresholds
    for entry_z in factors:
        for exit_z in factors:
            entry_long = data['Z-score'] < -entry_z
            entry_short = data['Z-score'] > exit_z
            exit_long_short = abs(data['Z-score']) < 0.01

            portfolio = vbt.Portfolio.from_signals(
                close=data[col],
                entries=entry_long,
                short_entries=entry_short,
                exits=exit_long_short,
                short_exits=exit_long_short,
                freq='1D'
            )

            sharpe_ratio = portfolio.sharpe_ratio().mean()
            total_return = portfolio.total_return().mean()
            max_drawdown = portfolio.max_drawdown().mean()

            # Save the results
            results.append({
                'Security': col,
                'Entry_Z': entry_z,
                'Exit_Z': exit_z,
                'Avg_Sharpe_Ratio': sharpe_ratio,
                'Avg_Return': total_return,
                'Max_Drawdown': max_drawdown
            })

# Convert results to DataFrame
result_df = pd.DataFrame(results)

# Display the results
print(result_df)


Processing security: S50H0^2
Processing security: S50M0^2
Processing security: S50U0^2
Processing security: S50Z0^2
Processing security: S50H1^2
Processing security: S50M1^2
Processing security: S50U1^2
Processing security: S50Z1^2
Processing security: S50H2^2
Processing security: S50M2^2
Processing security: S50U2^2
Processing security: S50Z2^2
Processing security: S50H3^2
Processing security: S50M3^2
Processing security: S50U3^2
Processing security: S50Z3^2
Processing security: S50H4^2
Processing security: S50M4^2
Processing security: S50U4
Processing security: S50Z4
    Security  Entry_Z  Exit_Z  Avg_Sharpe_Ratio  Avg_Return  Max_Drawdown
0    S50H0^2      0.5     0.5          4.822723    0.484696     -0.098789
1    S50H0^2      0.5     1.0          4.421964    0.436877     -0.098789
2    S50H0^2      0.5     1.5          3.533156    0.281405     -0.098789
3    S50H0^2      0.5     2.0          0.367275    0.011042     -0.176471
4    S50H0^2      0.5     2.5          1.708510    0.1

In [4]:
result_df.to_excel("TFEXS50.xlsx", index=False)